<a href="https://colab.research.google.com/github/RishabhMehra/DLCVNLP/blob/main/Assignmnet%203/ResNet_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Aug 29 16:22:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [3]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [4]:
num_classes = 100
nb_epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

#Pre-process the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
datagen.fit(x_train)

169017344/169001437 [==============================] - 11s 0us/step


In [5]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [6]:
resnet_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))


for layer in resnet_model.layers:
  if isinstance(layer, BatchNormalization):
    layer.trainable = True
  else:
    layer.trainable = False



model = Sequential()
#model.add(keras.Input(shape=(32,32,3)))
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(tf.keras.layers.experimental.preprocessing.Resizing(224, 224))
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(num_classes,activation='softmax'))

94781440/94765736 [==============================] - 3s 0us/step


In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit_generator(datagen.flow(x_train, y_train,batch_size=32), steps_per_epoch=50, epochs=20, validation_data=(x_test, y_test))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
50/50 [==============================] - 116s 2s/step - loss: 4.3634 - accuracy: 0.0769 - val_loss: 3.9058 - val_accuracy: 0.1373
Epoch 2/20
50/50 [==============================] - 88s 2s/step - loss: 3.5520 - accuracy: 0.1806 - val_loss: 3.2347 - val_accuracy: 0.2595
Epoch 3/20
50/50 [==============================] - 88s 2s/step - loss: 3.1254 - accuracy: 0.2706 - val_loss: 2.6253 - val_accuracy: 0.3561
Epoch 4/20
50/50 [==============================] - 88s 2s/step - loss: 2.7817 - accuracy: 0.3363 - val_loss: 2.3973 - val_accuracy: 0.3760
Epoch 5/20
50/50 [==============================] - 88s 2s/step - loss: 2.5317 - accuracy: 0.3862 - val_loss: 2.1558 - val_accuracy: 0.4369
Epoch 6/20
50/50 [==============================] - 89s 2s/step - loss: 2.4085 - accuracy: 0.4087 - val_loss: 1.8710 - val_accuracy: 0.4923
Epoch 7/20
50/50 [==============================] - 89s 2s/step - loss: 2.2630 - accuracy: 0.4350 - val_loss: 1.6642 - val_accuracy: 0.5487
Epoch 8/20
50/50 [=

In [ ]:
model.summary()

In [ ]:
model.summary()